In [4]:
## 발행정보
spread = 0.003    # 쿠폰에 더해지는 스프레드

reset_dates = ['2021-08-12','2021-11-15','2022-02-14','2022-05-13']     # 금리 결정일
begin_dates = ['2021-08-13','2021-11-16','2022-02-15','2022-05-16']     # 기산일
end_dates = ['2021-11-16','2022-02-15','2022-05-16','2022-08-17']       # 기말일
settle_dates = ['2021-11-16','2022-02-15','2022-05-16','2022-08-17']    # 쿠폰 지급일(결제일)
  
fixing_dates = ['2021-08-12']     # 금리 결정일 <= 평가일 <= 쿠폰지급일 인 경우 결제가 되지 않았기 때문에 사용
fixing_prices = [0.0072]          # 위와 같은 상황에서 금리값

## 마켓데이터 - 2021-11-05일자 IRSKRW_SPOT Curve
spot_tenors = [0.00278,0.25,0.5,0.75,1,1.25,1.5,1.75,2,2.25,2.5,2.75,3,3.25,3.5,3.75,4,4.25,4.5,4.75,5,5.25,5.5,5.75,6,
          6.25,6.5,6.75,7,7.25,7.5,7.75,8,8.25,8.5,8.75,9,9.25,9.5,9.75,10,12,15,20]
spot_rates = [0.0076,0.011284,0.013356,0.01468,0.016333,0.017209,0.018088,0.018741,0.019396,0.019557,0.019718,0.01988,
         0.020043,0.020046,0.02005,0.020054,0.020058,0.020036,0.020016,0.019995,0.019973,0.019955,0.019939,0.019922,
         0.019905,0.019887,0.01987,0.019851,0.019836,0.019827,0.019825,0.019825,0.019826,0.019826,0.019825,0.019821,
         0.019815,0.019807,0.019801,0.019799,0.019806,0.019942,0.019206,0.01779]

#계산일(평가일)
val_date = '2021-11-05'         


In [5]:
import datetime
import math

# 사용 할 함수 정의

# 보간법을 이용하여 지정된  tenor 사이의 teminate_time 의 이자율 산출
def get_ir_interpolation(target_tenors, target_rates, selected_tenor):
    if len(target_tenors) == 1:
        return target_rates[0]
    if target_tenors[-1] <= selected_tenor:
        return target_rates[-1]

    for i, tenor in enumerate(target_tenors):
        if tenor == selected_tenor:
            return target_rates[i]
        else:
            if tenor > selected_tenor:
                return target_rates[i-1] + (selected_tenor - target_tenors[i-1]) \
                       / (target_tenors[i] - target_tenors[i-1]) * (target_rates[i] - target_rates[i-1])


In [36]:
# 두 날짜 차이의 연환산값 산출
def get_year_diff_two_dates(start_date, end_date):
    start = datetime.datetime.strptime(start_date, "%Y-%m-%d")
    end = datetime.datetime.strptime(end_date, "%Y-%m-%d")
    return ((end - start).days) / 365

# 단순히 두 날짜의 차이 산출
def get_diff_two_dates(start_date, end_date):
    start = datetime.datetime.strptime(start_date, "%Y-%m-%d")
    end = datetime.datetime.strptime(end_date, "%Y-%m-%d")
    return ((end - start).days)

In [7]:
# 평가일(계산일) ~ 각 쿠폰일의 날짜 차이에 따른 할인 금리 만들기
discount_tenors = []
discount_rates = []

for date in settle_dates:
    discount_tenors.append(get_year_diff_two_dates(val_date,date))


In [15]:
for settle_date, tenor in zip(settle_dates, discount_tenors):
    print(f"결제일 {settle_date} ~ 평가일 {val_date} 의 연 환산 값 : {round(tenor, 10)}")

결제일 2021-11-16 ~ 평가일 2021-11-05 의 연 환산 값 : 0.0301369863
결제일 2022-02-15 ~ 평가일 2021-11-05 의 연 환산 값 : 0.2794520548
결제일 2022-05-16 ~ 평가일 2021-11-05 의 연 환산 값 : 0.5260273973
결제일 2022-08-17 ~ 평가일 2021-11-05 의 연 환산 값 : 0.7808219178


In [12]:
# 위에서 구한 discount_tenors 값에 따른 각각의 이자율 산출
for tenor in discount_tenors:
    discount_rates.append(get_ir_interpolation(target_tenors=spot_tenors, target_rates=spot_rates, selected_tenor=tenor))

In [17]:
for tenor, rate in zip(discount_tenors, discount_rates):
    print(f"tenor {round(tenor, 7)} 의 rate : {round(rate, 7)}")

tenor 0.030137 의 rate : 0.0080077
tenor 0.2794521 의 rate : 0.0115281
tenor 0.5260274 의 rate : 0.0134938
tenor 0.7808219 의 rate : 0.0148838


* Discount Tenor 산출 공식 : Discount Factor $= exp(-rate * tenor)$


In [30]:
# 엑셀 시트 ANS 탭의 DiscountFactor부분
discount_factor = []
for tenor, rate in zip(discount_tenors, discount_rates):
    discount_factor.append(math.exp(- rate * tenor))

In [31]:
for i, df in enumerate(discount_factor):
    print(f"{i} - th discount factor : {df}")

0 - th discount factor : 0.9997587022026649
1 - th discount factor : 0.9967836327769407
2 - th discount factor : 0.9929270021157247
3 - th discount factor : 0.9884456768834075


In [132]:
# forward rate 산출
# 각각의 선도 이자율의 의미 : 평가일 시점에서 파악한 i번째 기산일 ~ i번째 기말일까지의 이자율

In [21]:
coupon_list = []
coupon_num = len(settle_dates)

for i in range(coupon_num):
    if get_year_diff_two_dates(val_date, settle_dates[i]) <= 0:    # 쿠폰지급일 <= 평가일 인 경우 쿠폰 0
        coupon_list.append(0) 
    elif get_year_diff_two_dates(reset_dates[i], val_date) >= 0:   # 리셋일 <= 평가일 <= 쿠폰지급일 인 경우 이미 리셋일에 정해진 이자가 fixing_prices에 있기 때문에 그것을 그대로 가져옴
        idx = fixing_dates.index(reset_dates[i])
        coupon_list.append(fixing_prices[idx])
    else:                                                             # 평가일 < 리셋일 인 경우 Forward Rate 를 산출
        coupon_list.append((discount_factor[i-1] / discount_factor[i] - 1) 
            / (get_diff_two_dates(settle_dates[i-1], settle_dates[i]) / 365))


In [32]:
# 엑셀 시트 ANS 탭의 확정(추정)금리 부분
# 각각의 금리는 연 환산이므로 실제로 지급하는 쿠폰은 "기말일-기산일" 로 나눠주어야 한다. 
print("각 쿠폰일의 추정 금리")
for i, fr in enumerate(coupon_list):
    print(f"{i} - th forward rate : {fr}")    

각 쿠폰일의 추정 금리
0 - th forward rate : 0.0072
1 - th forward rate : 0.011971475434708705
2 - th forward rate : 0.015752195146766425
3 - th forward rate : 0.017793589765074076


In [23]:
# 확정(추정)금리가 연이자율이기 때문에 기산일~기말일 사이로 바꿔주기 위해서 곱할 것들을 만들어줌
begin_end_diff = []
for begin, end in zip(begin_dates, end_dates):
    begin_end_diff.append(get_year_diff_two_dates(begin, end))

In [25]:
# 엑셀 시트 ANS 탭의 지급 쿠폰 부분
pay_cpn = []
for i in range(len(coupon_list)):
    pay_cpn.append(0)

for i, coupon in enumerate(coupon_list):
    if coupon == 0:
        pay_cpn[i] = 0
    else:
        pay_cpn[i] = (coupon + spread) * begin_end_diff[i]    # 연이율에서 기산일-기말일 로 바꿔줌


In [34]:
for i, act_cpn_rate in enumerate(pay_cpn):
    print(f"{i} - th Pay Coupon rate : {act_cpn_rate} at settle date {settle_dates[i]}")

0 - th Pay Coupon rate : 0.0026547945205479453 at settle date 2021-11-16
1 - th Pay Coupon rate : 0.003732614423447924 at settle date 2022-02-15
2 - th Pay Coupon rate : 0.00462382894029857 at settle date 2022-05-16
3 - th Pay Coupon rate : 0.00529809273466271 at settle date 2022-08-17


In [27]:
result = 0
for i in range(len(pay_cpn)):
    if i < (len(pay_cpn) -1):
        result += pay_cpn[i] * discount_factor[i]
    else:
        result += (1 + pay_cpn[i]) * discount_factor[i]

In [35]:
print(f"Given FRN Price is {result}")

Given FRN Price is 1.0046484412399364


In [37]:
value_from_kiwoom_calculation = 1.00464844123994

In [38]:
print(f"같은 상품의 키움 계산기 값과 파이썬 값 차이 : {result - value_from_kiwoom_calculation}")

같은 상품의 키움 계산기 값과 파이썬 값 차이 : -3.552713678800501e-15
